<a href="https://colab.research.google.com/github/MathewBiddle/sandbox/blob/main/notebooks/CKAN_API_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Search [IOOS Data Catalog](https://data.ioos.us/) for CF standard names that contain `oxygen` and `sea_water`.

In [1]:
try:
   import ckanapi
except:
   !pip install ckanapi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ckanapi: filename=ckanapi-4.7-py3-none-any.whl size=43289 sha256=7fb3f9236139ca7256aaabdf6b048468e348fc2948673ef7022b3f65844a9bf2
  Stored in directory: /root/.cache/pip/wheels/d1/07/e2/8237645142c5eb6502d5f1e7f97d78d721e9cd22a376dc78bc
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6409a076f7734291b07087c117ac2c96dcdb3ae7af2cfc2a8f7f170d7f31bbde
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built ckanapi docopt


Build CKAN API query.

In [2]:
from ckanapi import RemoteCKAN
ua = 'ckanapiioos/1.0 (+https://ioos.us/)'

#ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua)
ioos_catalog

Tell me what organizations are there.

In [3]:
orgs = ioos_catalog.action.organization_list()
print(orgs)

['aoos', 'caricoos', 'cdip', 'cencoos', 'comt', 'gcoos', 'glider-dac', 'glos', 'hf-radar-dac', 'ioos', 'maracoos', 'nanoos', 'neracoos', 'noaa-co-ops', 'noaa-ndbc', 'oceansites', 'pacioos', 'sccoos', 'secoora', 'unidata', 'usgs', 'us-navy']


Grab all the datasets available.

In [4]:
#datasets = ioos_catalog.action.package_search(fq='+cf_standard_names:mass_concentration_of_oxygen_in_sea_water', rows=50)
datasets = ioos_catalog.action.package_search()
datasets['count']

37111

Collect CF standard names that contain `oxygen` and `sea_water` from the CF standard name list.

In [5]:
import pandas as pd

url = "https://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml"

df = pd.read_xml(url, xpath="entry")

std_names = df.loc[(df['id'].str.contains('oxygen') & df['id'].str.contains('sea_water'))]

std_names

,id,canonical_units,grib,amip,description
458,depth_at_shallowest_local_minimum_in_vertical_...,m,None,None,Depth is the vertical distance below the surfa...
606,fractional_saturation_of_oxygen_in_sea_water,1,None,None,Fractional saturation is the ratio of some mea...
1319,mass_concentration_of_oxygen_in_sea_water,kg m-3,None,None,Mass concentration means mass per unit volume ...
1667,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,Mole concentration means number of moles per u...
1668,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,"""Mole concentration at saturation"" means the m..."
1669,mole_concentration_of_dissolved_molecular_oxyg...,mol m-3,None,None,Mole concentration means number of moles per u...
1918,moles_of_oxygen_per_unit_mass_in_sea_water,mol kg-1,None,None,"moles_of_X_per_unit_mass_inY is also called ""m..."
3085,surface_molecular_oxygen_partial_pressure_diff...,Pa,None,None,"The surface called ""surface"" means the lower b..."
3578,temperature_of_sensor_for_oxygen_in_sea_water,K,None,None,Temperature_of_sensor_for_oxygen_in_sea_water ...
4583,volume_fraction_of_oxygen_in_sea_water,1,None,None,"""Volume fraction"" is used in the construction ..."


Search IOOS Data Catalog for CF standard names that match those above.

In [122]:
from ckanapi import RemoteCKAN
import time
ua = 'ckanapiioos/1.0 (+https://ioos.us/)'

#ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua)
ioos_catalog

df_out = pd.DataFrame()

for std_name in std_names['id']:

  print(std_name)

  fq = '+cf_standard_names:{}'.format(std_name)

  result_count = 0

  while True:
    datasets = ioos_catalog.action.package_search(fq=fq, rows=500)

    num_results = datasets['count']

    print(f"num_results: {num_results}, result_count: {result_count}")

    for dataset in datasets['results']:
        #print(dataset['title'])
        df = pd.DataFrame({'title': [dataset['title']],
                         'url': [dataset['resources'][0]['url']],
                         'org': [dataset['organization']['title']]})

        df_out = pd.concat([df_out, df])

        result_count = result_count + 1

    time.sleep(1)

    if(result_count >= num_results):
        print(f"num_results: {num_results}, result_count: {result_count}")
        break


  # print('found {} dataset(s)'.format(datasets['count']))
  # print('\tcollecting {} datasets'.format(len(datasets['results'])))

  # if datasets['count'] > 0:
  #   for dataset in datasets['results']:
  #     #print('\t',dataset['title'],dataset['resources'][0]['url'])

  #     df = pd.DataFrame({'title': [dataset['title']],
  #                        'url': [dataset['resources'][0]['url']],
  #                        'org': [dataset['organization']['title']]})

  #     #print('\tcollected {} datasets'.format(len(df)))

  #     df_out = pd.concat([df_out, df])

df_out

depth_at_shallowest_local_minimum_in_vertical_profile_of_mole_concentration_of_dissolved_molecular_oxygen_in_sea_water
num_results: 0, result_count: 0
num_results: 0, result_count: 0
fractional_saturation_of_oxygen_in_sea_water
num_results: 4717, result_count: 0
num_results: 4717, result_count: 500
num_results: 4717, result_count: 1000
num_results: 4717, result_count: 1500
num_results: 4717, result_count: 2000
num_results: 4717, result_count: 2500
num_results: 4717, result_count: 3000
num_results: 4717, result_count: 3500
num_results: 4717, result_count: 4000
num_results: 4717, result_count: 4500
num_results: 4717, result_count: 5000
mass_concentration_of_oxygen_in_sea_water
num_results: 3890, result_count: 0
num_results: 3890, result_count: 500
num_results: 3890, result_count: 1000
num_results: 3890, result_count: 1500
num_results: 3890, result_count: 2000
num_results: 3890, result_count: 2500
num_results: 3890, result_count: 3000
num_results: 3890, result_count: 3500
num_results: 389

,title,url,org
0,"Walton-Smith CTD, WS22281, WS22281_StnGlider, ...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS23061, WS23061_WS23061_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22215, WS22215_2022_08_Wea...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22281, WS22281_StnBG2, 202...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22215, WS22215_2022_08_Wea...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
...,...,...,...
0,"Walton-Smith CTD, WS1015, WS1015_015_5, 2010-0...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0718, WS0718_WS0718_007, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0618, WS0618_WS0618_001, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0623, WS0623_WS0623_17, 20...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS


How many orgs have dissolved O2 data?

In [126]:
df_out.groupby(by='org').count()

,title,url
org,,
AOOS,166,166
CeNCOOS,12,12
GCOOS,6823,6823
Glider DAC,5260,5260
MARACOOS,17,17
NANOOS,3,3
NERACOOS,361,361
SCCOOS,9,9
SECOORA,1280,1280


How many of those are unique? The search above looked for all cf standard names for dissolved O2. We might have some duplicate returns, so lets get rid of endpoints that are the same.

In [127]:
df_group = df_out.groupby(by='url').count()

df_group

,title,org
url,,
http://erddap.maracoos.org/erddap/tabledap/ru05-20150105T1600-profile-sci-delayed,2,2
http://erddap.maracoos.org/erddap/tabledap/ru05-20150115T1443-profile-sci-delayed,2,2
http://erddap.maracoos.org/erddap/tabledap/ru05-20150201T2130-profile-sci-delayed,2,2
http://erddap.maracoos.org/erddap/tabledap/ru30-20180502T1355-profile-sci-delayed,1,1
http://erddap.maracoos.org/erddap/tabledap/ru30-20190201T1712-profile-sci-delayed,1,1
...,...,...
https://gliders.ioos.us/thredds/dodsC/deployments/rutgers/um_242-20210630T1916/um_242-20210630T1916.nc3.nc,2,2
https://gliders.ioos.us/thredds/dodsC/deployments/rutgers/um_242-20211105T1601/um_242-20211105T1601.nc3.nc,2,2
https://gliders.ioos.us/thredds/dodsC/deployments/rutgers/um_242-20220513T1603/um_242-20220513T1603.nc3.nc,2,2


In [129]:
df_out_nodups = df_out.drop_duplicates(subset=['url'],keep='last')

df_out_nodups

,title,url,org
0,"Walton-Smith CTD, WS23061, WS23061_WS23061_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS23010, WS23010_WS23011_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS23010, WS23010_WS23011_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
...,...,...,...
0,"Walton-Smith CTD, WS1015, WS1015_015_5, 2010-0...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0718, WS0718_WS0718_007, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0618, WS0618_WS0618_001, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0623, WS0623_WS0623_17, 20...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS


Just to see what endpoints we're working with, let's show endpoints that aren't ERDDAP.

In [130]:
df_out_nodups.loc[~df_out_nodups['url'].str.contains('erddap')]

,title,url,org
0,ce_383-20170719T1900-delayed,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ce_327-20180401T1431-delayed,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ce_384-20171208T1704-delayed,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ce_312-20190515T1750-delayed,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ce_311-20161011T2101-delayed,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
...,...,...,...
0,um_242-20220513T1603,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ru32-20211103T1405,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,electa-20210827T1516,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC
0,ru36-20220223T1807,https://gliders.ioos.us/thredds/dodsC/deployme...,Glider DAC


Glider DAC is already making it to NCEI, so we can drop those entries.

In [131]:
df_out_nodups_no_glider = df_out_nodups.loc[~df_out_nodups['org'].str.contains('Glider DAC')]
df_out_nodups_no_glider

,title,url,org
0,"Walton-Smith CTD, WS23061, WS23061_WS23061_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS23010, WS23010_WS23011_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS23010, WS23010_WS23011_Kel...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
...,...,...,...
0,"Walton-Smith CTD, WS1015, WS1015_015_5, 2010-0...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0718, WS0718_WS0718_007, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0618, WS0618_WS0618_001, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS
0,"Walton-Smith CTD, WS0623, WS0623_WS0623_17, 20...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS


What's the remaining distribution?

In [132]:
df_out_nodups_no_glider.groupby(by='org').count()

,title,url
org,,
AOOS,13,13
CeNCOOS,11,11
GCOOS,837,837
MARACOOS,11,11
NANOOS,2,2
NERACOOS,29,29
SCCOOS,2,2
SECOORA,132,132


Let's print out the response as a csv.

In [133]:
print(df_out_nodups_no_glider.to_csv(index=False))

title,url,org
"Walton-Smith CTD, WS23061, WS23061_WS23061_Kelble_Stn.V3, 2023-03-05, 27.0308N 82.619W",https://gcoos5.geos.tamu.edu/erddap/tabledap/WS23061_WS23061_WS23061_Kelble_Stn_V3,GCOOS
"Walton-Smith CTD, WS23010, WS23010_WS23011_Kelble_Stn.018, 2023-01-11, 24.5972N 81.1835W",https://gcoos5.geos.tamu.edu/erddap/tabledap/WS23010_WS23010_WS23011_Kelble_Stn_018,GCOOS
"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn.WS, 2022-12-08, 24.4758N 81.7177W",https://gcoos5.geos.tamu.edu/erddap/tabledap/WS22337_WS22337_WS22337_Stn_WS,GCOOS
"Walton-Smith CTD, WS23010, WS23010_WS23011_Kelble_Stn.064, 2023-01-16, 25.1657N 81.1612W",https://gcoos5.geos.tamu.edu/erddap/tabledap/WS23010_WS23010_WS23011_Kelble_Stn_064,GCOOS
"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn.V9, 2022-12-06, 26.7295N 83.1032W",https://gcoos5.geos.tamu.edu/erddap/tabledap/WS22337_WS22337_WS22337_Stn_V9,GCOOS
"Walton-Smith CTD, WS22337, WS22337_WS22337_Stn.CW3, 2022-12-07, 28.0103N 82.9595W",https://gcoos5.geos.tamu.edu/erd